In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB

In [ ]:
pip install datasets

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [ ]:
from datasets import load_dataset

#load the test data for the cross-model evaluation of the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "cross_model_evaluation")

print(data.keys())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating cross_model_evaluation_de_en split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_en_de split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_fr_en split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating cross_model_evaluation_en_fr split:   0%|          | 0/600 [00:00<?, ? examples/s]

dict_keys(['cross_model_evaluation_de_en', 'cross_model_evaluation_en_de', 'cross_model_evaluation_fr_en', 'cross_model_evaluation_en_fr'])


In [ ]:
data_de_en = data['cross_model_evaluation_de_en'].to_pandas()
data_en_de = data['cross_model_evaluation_en_de'].to_pandas()
data_fr_en = data['cross_model_evaluation_fr_en'].to_pandas()
data_en_fr = data['cross_model_evaluation_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

,id,source,hyp1,hyp2
0,0,"""Perry erklärte, er werde „nach Texas zurückke...",Perry declared that he would'return to Texas t...,Perry declared that he would return to Texas t...
1,1,"Bachmann, die im August die Ames-Probeumfrage ...","Bachmann, who won the Ames straw poll in Augus...","Bachmann, who won the Ames straw poll in Augus..."
2,2,Die Kruste ist auf der nahen Seite etwa 70 km ...,The crust is on the nearby side about 70 km th...,The crust is approximately 70 km thick on the ...
3,3,Der Anwalt forderte die Wirtschaftsprüferin au...,The lawyer demanded that the auditor provide d...,The lawyer demanded the auditor to present the...
4,4,"Diese Art ist in der Schweiz, in Frankreich un...",This type of art is represented in Switzerland...,This type of art is represented in Switzerland...


,id,source,hyp1,hyp2
0,0,The pirates were attacking the ship in order t...,"Die Piraten griffen das Schiff an, um die Prin...","Die Piraten griffen das Schiff an, um die Prin..."
1,1,"Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, ein Mathematiker und Informatik..."
2,2,In the summer of 1924 he went to Smackover in ...,Im Sommer 1924 machte er Urlaub in Smackover i...,Im Sommer 1924 ging er nach Smackover in Süd-A...
3,3,I am informed that yesterday evening a frigate...,"1. Ich bin informiert, dass gestern Abend ein ...","Ich erfuhr, dass gestern Abend ein Fregattenbo..."
4,4,"Heels should be low and wide. Sand, gravel or ...","Absätze sollten niedrig und breit sein. Sand, ...",Absätze sollten sehr niedrig und sehr breit se...


## Load Model

In [ ]:
#define the Model
model = "google/gemma-7b-it"
# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallucination(source, hyp1, hyp2, translation_language)
                print("source: ", source)
                print("hyp1: ", hyp1)
                print("hyp2: ", hyp2)
                print("result: ", result)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

#English-German

In [ ]:
data_en_de.head()

,id,source,hyp1,hyp2
0,0,The pirates were attacking the ship in order t...,"Die Piraten griffen das Schiff an, um die Prin...","Die Piraten griffen das Schiff an, um die Prin..."
1,1,"Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, a mathematician and computer sc...","Curtis Cooper, ein Mathematiker und Informatik..."
2,2,In the summer of 1924 he went to Smackover in ...,Im Sommer 1924 machte er Urlaub in Smackover i...,Im Sommer 1924 ging er nach Smackover in Süd-A...
3,3,I am informed that yesterday evening a frigate...,"1. Ich bin informiert, dass gestern Abend ein ...","Ich erfuhr, dass gestern Abend ein Fregattenbo..."
4,4,"Heels should be low and wide. Sand, gravel or ...","Absätze sollten niedrig und breit sein. Sand, ...",Absätze sollten sehr niedrig und sehr breit se...


In [ ]:
def detect_hallucination(source, hyp1, hyp2, translation_language):

  answer_format = {"label": " ",
                   "type": " "}

  # prompt for translation generation task
  user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to the language of "src".
    Then, compare these translations with "src" to detect which of the two hypotheses is hallucinated ("label") and what type of hallucination ("type") it is.
    Provide the result in the following format: {answer_format}.

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  label = outputs[0]["generated_text"][len(prompt):]
  #print(label)

  return label

# Englisch - Deutsch

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_de.csv"
skipped_rows, df = save_trial_results(data_en_de, csv_filename, "German")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Translation:**

* **hyp1:** Die Piraten griffen das Schiff an, um die Prinzessin und ihre Schatzkiste gefangen zu nehmen.
Translation: The pirates were attacking the ship in order to capture the princess and her treasure chest.

* **hyp2:** Die Piraten griffen das Schiff an, um die Prinzessin gefangen zu nehmen.
Translation: The pirates were attacking the ship in order to capture the princess.

**Comparison:**

The translation of "hyp1" is very similar to the original text "src", except for the addition of the phrase "und ihre Schatzkiste gefangen zu nehmen". This phrase is not present in the original text.

The translation of "hyp2" is also very similar to the original text, but it lacks the phrase "und ihre Schatzkiste gefangen zu nehmen".

**Result:**

{'label': 'hyp1', 'type': 'Overstatement'}

The hallucination is the addition of the phrase
source:  The pirates were attacking the ship in order to capture the princess.
hyp1:  Die Piraten griffen das Schiff an, um die Prinzessin u

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Translation:**

**src:** Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25.

**hyp1:** Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25th. (correct)

**hyp2:** Curtis Cooper, ein Mathematiker und Informatikprofessor an der University of Central Missouri, hat am 25. Januar die größte bekannte Primzahl entdeckt.

**Result:**

{'label': 'hyp2', 'type': 'grammatical'}
source:  Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25.
hyp1:  Curtis Cooper, a mathematician and computer science professor at the University of Central Missouri, has discovered the largest known prime number to date on January 25th. (correct)
hyp2:  Curtis Cooper, ei

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

**Translation:**

**hyp1:** The Onion has become a successful news parody empire with a large following.
**hyp2:** The Onion has become a powerful news organization with a significant online presence.

**Comparison:**

The translation of "hyp1" is very similar to the original text "src", but it lacks the specific details about the Onion's success and its various platforms. The translation of "hyp2" is more accurate in terms of the Onion's actual size and reach, but it does not capture the humorous tone of the original text.

**Result:**

{'label': 'hyp2', 'type': 'overgeneralization'}
source:  Since its inception, The Onion has become a veritable news parody empire, with a print edition, a website that drew 5,000,000 unique visitors in the month of October, personal ads, a 24 hour news network, podcasts, and a recently launched world atlas called Our Dumb World.
hyp1:  The Onion has become a successful news parody empire with a large following.
hyp2:  The Onion has become a powerful ne

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Hyp1:**
"The phone rang, and I managed to pick it up, and it was my father, and I said, ""I'm in serious trouble. We need to do something.""

**Hyp2:**
"After four hours of lying and staring at the phone, it rang, and I managed to pick it up, and it was my father, and I said, ""I'm in serious trouble. We need to do something.""

**Result:**
{'label': 'hyp1', 'type': 'overstatement'}

**Explanation:**

* **Translation:**
    * Hyp1 is a faithful translation of the source text, except for the omission of the phrase "after four full hours of my lying and staring at it".
    * Hyp2 is a rephrased version of the source text that includes the phrase "after four full hours of lying and staring at the phone".

* **Comparison:**
    * Hyp1 is
source:  "And finally, after four full hours of my lying and staring at it, the phone rang, and somehow I managed to pick it up, and it was my father, and I said, ""I'm in serious trouble. We need to do something."""
hyp1:  ..............................

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Result

```
{'label': 'hyp1', 'type':'synonym'}
```

**Explanation:**

- **Hyp1:** "Die Piraten attackierten das Schiff, um die Prinzessin zu fangen" is a translation of "The pirates were attacking the ship in order to capture the princess" that is a synonym of the original text.
- **Hyp2:** "Die Piraten attackierten das Schiff, um die Prinzessin zu fangen" is also a translation of the original text, but it uses different phrasing and word choice.

Therefore, the result is:

```
{'label': 'hyp1', 'type':'synonym'}
```
source:  The pirates were attacking the ship in order to capture the princess.
hyp1:  Die Piraten attackierten das Schiff, um die Prinzessin zu fangen.
hyp2:  Die Piraten attackierten das Schiff, um die Prinzessin zu fangen.
result:  ## Result

```
{'label': 'hyp1', 'type':'synonym'}
```

**Explanation:**

- **Hyp1:** "Die Piraten attackierten das Schiff, um die Prinzessin zu fangen" is a translation of "The pirates were attacking the ship in order to capture the princ

In [ ]:
skipped_rows

# German - English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_de_en.csv"
skipped_rows_de_en, df = save_trial_results(data_de_en, csv_filename, "English")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  "Perry erklärte, er werde „nach Texas zurückkehren, um das Ergebnis der Vorwahl von heute Abend auszuwerten und zu entscheiden, ob es in diesem Rennen einen weiterführenden Weg für mich gibt"", aber sagte später, er werde im Rennen bleiben und bei den Vorwahlen in South Carolina am 21. Januar antreten."
hyp1:  Perry declared that he would'return to Texas to evaluate the result of tonight's primary and decide whether there is a further way for me in this race', but later said he would stay in the race and participate in the primaries in South Carolina on January 21.
hyp2:  Perry declared that he would return to Texas to evaluate the result of tonight's primary and decide whether there is a further way for him in this race, but later said he would stay in the race and participate in the South Carolina primary on January 21, and then consider running for president.
result:  ## Result

{'label': 'hyp2', 'type': 'Overgeneralization'}

**Explanation:**

* **Hyp1:** Translates "hyp1"

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Bachmann, die im August die Ames-Probeumfrage gewann, beschloss, ihren Wahlkampf zu beenden.
hyp1:  Bachmann, who won the Ames straw poll in August, decided to end her campaign.
hyp2:  Bachmann, who won the Ames straw poll in August, decided to end her campaign.
result:  **Result:**

{'label': 'hyp1', 'type': 'Word-Order Hallucination'}

**Explanation:**

* **Translation of hyp1:** Bachmann, who won the Ames straw poll in August, decided to end her campaign.
* **Translation of hyp2:** Bachmann, who won the Ames straw poll in August, decided to end her campaign.
* **Comparison with src:** The translation of hyp1 is identical to the source text, while the translation of hyp2 differs in the word order.
* **Hallucination type:** Word-Order Hallucination
source:  Die Kruste ist auf der nahen Seite etwa 70 km dick und auf der fernen Seite 100 km.
hyp1:  The crust is on the nearby side about 70 km thick and on the far side 1000 km.
hyp2:  The crust is approximately 70 km thick on the

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

source:  Alton liegt am Missouri River oberhalb der Mündung des Mississippi.
hyp1:  Alton is located above the confluence of the Mississippi River and the Missouri River.
hyp2:  Alton is located below the confluence of the Mississippi River and the Missouri River.
result:  **Translation:**

**src:** Alton liegt am Missouri River oberhalb der Mündung des Mississippi.

**hyp1:** Alton is located above the confluence of the Mississippi River and the Missouri River.

**hyp2:** Alton is located below the confluence of the Mississippi River and the Missouri River.

**Result:**

{'label': 'hyp1', 'type': 'overestimation'}
source:  Seit seiner Gründung hat sich The Onion zu einem wahren Nachrichtenparodie-Imperium entwickelt, mit einer Printausgabe, einer Website, die im Oktober 5 000 000 Einzelbesucher hatte, personalisierten Werbeanzeigen, einem 24-Stunden-Nachrichtensender, Podcasts und einem kürzlich veröffentlichten Weltatlas mit dem Titel Our Dumb World.
hyp1:  The Onion has grown into a

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Der Designer wollte den Verkäufer beeindrucken.
hyp1:  The designer wanted to impress the salesperson.
hyp2:  The designer wanted to disappoint the salesperson.
result:  **Result:**
{'label': 'hyp1', 'type': 'overgeneralization'}

**Explanation:**

* **Translation of hyp1:** The designer wanted to impress the salesperson. This translation is accurate and captures the main idea of the source text.
* **Translation of hyp2:** The designer wanted to disappoint the salesperson. This translation is inaccurate and does not capture the main idea of the source text.
* **Comparison with src:** The translation of hyp1 is more similar to the source text than the translation of hyp2.
* **Hallucination:** Hyp2 is hallucinated because it is not supported by the evidence in the source text.
* **Type of hallucination:** Overgeneralization is the type of hallucination in this case.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.
hyp1:  The epicenter of the earthquake was approximately 20 kilometers (15 miles) north-northeast of Dillon and approximately 65 kilometers (40 miles) south of Butte.
hyp2:  The epicenter of the earthquake was approximately 20 kilometers (15 miles) south-southwest of Dillon and approximately 65 kilometers (40 miles) north of Butte.
result:  **Translation:**

**hyp1:**
The epicenter of the earthquake was approximately 20 kilometers (15 miles) north-northeast of Dillon and approximately 65 kilometers (40 miles) south of Butte.

**hyp2:**
The epicenter of the earthquake was approximately 20 kilometers (15 miles) south-southwest of Dillon and approximately 65 kilometers (40 miles) north of Butte.

**Comparison:**

The translation of hyp1 is identical to the source text, "src". Therefore, hyp1 is not hallucinated.

The translation of hyp2 is the mirr

In [ ]:
skipped_rows_de_en

# English-French

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_fr.csv"
skipped_rows_en_fr, df = save_trial_results(data_en_fr, csv_filename, "French")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


source:  Thus, a backcountry permit for any start date in May becomes available on 1 Jan.
hyp1:  Donc, un permis de randonnée pour tout départ en mai devient disponible le 1er mai.
hyp2:  Donc, un permis de randonnée pour tout début de mois en mai devient disponible le 1er janvier.
result:  **Result:**

{'label': 'hyp2', 'type': 'Date hallucination'}

**Explanation:**

* **Translation of hyp1:** "Un permis de randonnée pour tout départ en mai devient disponible le 1er mai."
* **Translation of hyp2:** "Un permis de randonnée pour tout début de mois en mai devient disponible le 1er janvier."
* **Comparison with src:** The translation of hyp2 differs from the source text "Thus, a backcountry permit for any start date in May becomes available on 1 Jan." in the month of the permit availability. The source text specifies "May," while the translation of hyp2 incorrectly states "January."
* **Hallucination type:** Date hallucination

Therefore, the result is: {'label': 'hyp2', 'type': 'Date ha

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  He was selected by the Philadelphia 76ers in the 77th round (7th pick overall) of the 1967 NBA Draft.
hyp1:  Il a été sélectionné par les 76ers de Philadelphie dans la 77ème tour (7ème choix global) de la draft NBA 1967.
hyp2:  Il a été sélectionné par les 76ers de Philadelphie dans la 77e tour (7e choix overall) de la draft NBA 1967.
result:  **Result:**

{'label': 'hyp2', 'type': 'over-specification'}

**Explanation:**

* **Translation of hyp1:** "Il a été sélectionné par les 76ers de Philadelphie dans la 77ème tour (7ème choix global) de la draft NBA 1967." This translation is accurate, except for the use of the phrase "choix global."
* **Translation of hyp2:** "Il a été sélectionné par les 76ers de Philadelphie dans la 77e tour (7e choix overall) de la draft NBA 1967." This translation is inaccurate, as the correct phrase is "7ème choix global."
* **Comparison with src:** The translation of hyp1 is more similar to the original text than the translation of hyp2. This is bec

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

source:  The issue started on January 1st when dozens of local residents started complaining to the Obanazawa Post Office that they hadn't received their traditional and regular New Year cards.
hyp1:  Le problème a commencé le 1er janvier lorsque des dizaines de résidents locaux ont déposé une plainte à la poste d'Obanazawa pour dire qu'ils n'avaient pas reçu leurs cartes de Noël traditionnelles et régulières.
hyp2:  Le problème a commencé le 1er janvier, lorsque des dizaines de résidents locaux ont déposé une plainte à la Poste d'Obanazawa pour dire qu'ils n'avaient pas reçu leurs cartes de la Saint-Sylvestre traditionnelles et régulières.
result:  ## Result

{'label': 'hyp1', 'type': 'Over-literal translation'}

The text "src" describes the issue of not receiving New Year cards. "hyp1" is a translation that is very close to "src", but uses the incorrect term "cartes de Noël" instead of "cartes de New Year". "hyp2" is a hallucination that completely misinterprets the text and is unrel

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  21-year-old Jesus joined Manchester City last year in January 2017 from Brazilian club Palmeiras for a reported fee of £27 million.
hyp1:  Le jeune Jésus a été vendu à Manchester City il y a deux ans pour 30 millions de dollars.
hyp2:  Le jeune joueur de 21 ans, Jésus, a rejoint Manchester City il y a un an, en janvier 2017, après avoir quitté le club brésilien Palmeiras pour une somme de 27 millions de livres sterling.
result:  **Result:**

{'label': 'hyp1', 'type': 'overt'}

**Explanation:**

* **Translation of hyp1:** "Le jeune Jésus a été vendu à Manchester City il y a deux ans pour 30 millions de dollars."
This translation is incorrect as it uses the incorrect currency ("dollars") and the word "vendu" instead of "acheté".

* **Comparison with src:** The translation of hyp1 differs from the original text in several ways, including the use of incorrect currency, the incorrect tense, and the use of the word "vendu".

* **Hallucination:** Hyp1 is hallucinated as it is not a f

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Of this population, 87.91% are ethnic Romanians, 7.53% are ethnic Hungarians and 4.43% ethnic Romani.
hyp1:  87,91% de cette population sont Roumains ethniques, 7,53% sont Hongrois ethniques et 4,43% sont Romanis.
hyp2:  Of this population, 87.91% are ethnic French, 7.53% are ethnic Germans and 4.43% ethnic Gypsies.
result:  **Translation:**

**hyp1:**
87,91% de cette population sont Roumains ethniques, 7,53% sont Hongrois ethniques et 4,43% sont Romanis.

**hyp2:**
87,91% are ethnic French, 7,53% are ethnic Germans and 4,43% are ethnic Gypsies.

**Result:**

{'label': 'hyp2', 'type': 'Overgeneralization'}
source:  Like many ethnic communities in Uganda, including the Langi, Acholi, and Alur, the Luo do not practice the ritual circumcision of males as initiation.
hyp1:  Les Luo, comme beaucoup de communautés ethniques en Ouganda, notamment les Langi, les Acholi et les Alur, ne pratiquent pas la circoncision masculine en tant qu'initiation.
hyp2:  Les Luo, comme beaucoup de com

In [ ]:
skipped_rows_en_fr

[18, 24, 158, 268, 282, 350, 397, 402, 433, 465, 529, 550, 589]

## French-English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_fr_en.csv"
skipped_rows_fr_en, df = save_trial_results(data_fr_en, csv_filename, "English")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


source:  David Allan Coe a rejoint le groupe de tournées et d'enregistrement de Warren Haynes en 1980, à l'âge de 20 ans.
hyp1:  David Allan Coe joined Warren Haynes' touring and recording group in 1980 at the age of 20.
hyp2:  David Allan Coe joined Warren Haynes' touring and recording group in 1979 at the age of 20.
result:  **Result:**

{'label': 'hyp2', 'type': 'Year Hallucination'}

**Explanation:**

* **Translation of hyp1:** "David Allan Coe joined Warren Haynes' touring and recording group in 1980 at the age of 20."

* **Translation of hyp2:** "David Allan Coe joined Warren Haynes' touring and recording group in 1979 at the age of 20."

* **Comparison with src:** The translation of hyp1 is identical to the source text, while the translation of hyp2 differs in the year.

* **Hallucination type:** Year Hallucination

* **Label:** hyp2
source:  Kabir Suman a enregistré plusieurs albums sous le nom de Suman Chattopaddhyay ou Suman Chatterjee entre 1992 et 1999.
hyp1:  Kabir Suman h

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Le problème a commencé le 1er janvier lorsque des dizaines de résidents locaux ont commencé à se plaindre au bureau de poste d'Obanazawa qu'ils n'avaient pas reçu leurs cartes de nouvel an traditionnelles et régulières.
hyp1:  The problem started on January 1st when dozens of local residents complained to the Obanazawa post office that they had not received their traditional and regular New Year's cards.
hyp2:  The problem started on January 1st when dozens of local residents complained at the Obanazawa post office that they had not received their traditional and regular New Year's cards.
result:  ## Result:

{'label': 'hyp2', 'type': 'Grammatical error'}

**Explanation:**

- **Hyp1:** Translation is accurate and matches the language of "src" exactly.
- **Hyp2:** Translation is grammatically incorrect. It uses the verb "complained at" instead of "complained to." This error is a common one and does not significantly alter the meaning of the sentence.

Therefore, "hyp2" is hallu

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

source:  Une biographie de Chris Sievey a été écrite par l'auteur de Manchester Mick Middles, et a été publiée en novembre 2014.
hyp1:  A biography of Chris Sievey has been written by Manchester author Mick Middles and was published in December 2014.
hyp2:  A biography of Chris Sievey has been written by Manchester author Mick Middles and was published in November 2014.
result:  **Result:**

{'label': 'hyp1', 'type': 'Date hallucination'}

**Explanation:**

* **Translation of hyp1:** A biography of Chris Sievey has been written by Manchester author Mick Middles and was published in December 2014.
* **Translation of hyp2:** A biography of Chris Sievey has been written by Manchester author Mick Middles and was published in November 2014.
* **Comparison with src:** The translation of hyp1 differs from the original text in the month of publication ("December" instead of "November").
* **Hallucination type:** Date hallucination.
source:  Nvidia a officiellement annoncé la NVIDIA TITAN V le 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Cette scène a été réécrite, bien que son récitatif initial sous forme découpée soit présent dans la première production.
hyp1:  This scene was rewritten, although its initial recitative in fragmented form is present in the first production.
hyp2:  The scene remained unchanged even though the initial recitative in fragmented form was present in the first production.
result:  **Translation:**

**src:** Cette scène a été réécrite, bien que son récitatif initial sous forme découpée soit présent dans la première production.

**hyp1:** This scene was rewritten, although its initial recitative in fragmented form is present in the first production.

**hyp2:** The scene remained unchanged even though the initial recitative in fragmented form was present in the first production.

**Result:**

{'label': 'hyp1', 'type': 'Overwriting'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


source:  Le format 35 mm est en fait, et c’est assez déconcertant, de 36 mm de largeur par 24 mm de hauteur.
hyp1:  The 35 mm format is actually smaller than 36 mm by 24 mm in size.
hyp2:  The 35 mm format is actually, and it is quite disconcerting, 36 mm wide by 24 mm high.
result:  **Translation:**

**hyp1:** The 35 mm format is actually smaller than 36 mm by 24 mm in size.
**hyp2:** The 35 mm format is actually, and it is quite disconcerting, 36 mm wide by 24 mm high.

**Comparison:**

The translation of "hyp1" is very similar to "src", except that it uses the phrase "smaller" instead of "lesser". The translation of "hyp2" is more accurate, but it includes the phrase "and it is quite disconcerting", which is not in "src".

**Result:**

{'label': 'hyp2', 'type': 'Hyperbole'}
source:  Le problème a commencé le 1er janvier lorsque des dizaines de résidents locaux ont commencé à se plaindre au bureau de poste d'Obanazawa qu'ils n'avaient pas reçu leurs cartes de nouvel an traditionnelle

In [ ]:
skipped_rows_fr_en

[4, 467, 476, 544]

## Convert to comma-separated csv

In [1]:
import pandas as pd
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_de_en_final.csv"


df = pd.read_csv(csv_filename, sep=";")



In [2]:
df.head()

,id,label
0,0,hyp2
1,1,hyp1
2,2,hyp1
3,3,hyp2
4,4,hyp2


In [3]:
len(df)

600

In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [5]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_fr_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp1


In [6]:
len(df)

600

In [7]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [8]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_fr_en_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp1
4,4,NaN


In [9]:
len(df)

600

In [10]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [11]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Cross_model_evaluation_task/Gemma/results_gemma_en_de_final.csv"


df = pd.read_csv(csv_filename, sep= ";")
df.head()


,id,label
0,0,hyp1
1,1,hyp2
2,2,hyp1
3,3,hyp2
4,4,hyp2


In [12]:
df.to_csv(csv_filename, mode='w', header=True, index=False)